In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import re

In [3]:
import pickle
with open('poetry_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [24]:
import string
def clean_text(text):
    if text.str.contains(any())
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = text.replace('\%','')
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(filter(lambda x:x[0]!="@", text.split()))
    return text

clean_corpus = list(map(clean_text, corpus))

In [25]:
print(clean_corpus[0])
print(clean_corpus[0].splitlines())
corpus_lines = [poem.splitlines() for poem in clean_corpus]

dear writers i’m compiling the first in what i hope is a series of publications i’m calling artists among artists the theme for issue is “faggot dinosaur” i hope to hear from you thank you and best wishes i think that i shall never fear a brontosaurus that is queer iguanodon as fetisheer a mammoth bringing up the rear an astrodon with extra gear metrosexual squirrel and deer a breeder with a dance career a fruit with cauliflower ear a lesbianic chanticleer a grinning limpishwristed lear the weird one or the mutineer but those who perfectly adhere stay clear stay clear stay clear stay clear
['dear writers i’m compiling the first in what i hope is a series of publications i’m calling artists among artists the theme for issue is “faggot dinosaur” i hope to hear from you thank you and best wishes i think that i shall never fear a brontosaurus that is queer iguanodon as fetisheer a mammoth bringing up the rear an astrodon with extra gear metrosexual squirrel and deer a breeder with a dance 

In [18]:
corpus_lines[0:6]

[['dear writers i’m compiling the first in what i hope is a series of publications i’m calling artists among artists the theme for issue is “faggot dinosaur” i hope to hear from you thank you and best wishes i think that i shall never feara brontosaurus that is queer iguanodon as fetisheera mammoth bringing up the rearan astrodon with extra gear metrosexual squirrel and deera breeder with a dance careera fruit with cauliflower ear a lesbianic chanticleera grinning limpishwristed learthe weird one or the mutineer but those who perfectly adherestay clear stay clear stay clear stay clear'],
 ['philosophicin its complex ovoid emptinessa skillful pundit coined it as a sortof stopgap doorstop for thosequaint equationsromans neverdreamt of in form completely cleverand discrete—a mirror come unsilveredloose watch face without the worksa hollowed globefrom tip to toeunbroken it evades the grapplinghooks of mass tilts the thin rim of no thingremains embryonic sumnoncogito'],
 ['wed like to talk 

In [26]:
sum(map(len, corpus_lines))/float(len(corpus_lines))

0.9998722208024534

In [28]:
while not isinstance(corpus_lines[0], str):
    corpus_lines = [line for poem in corpus_lines for line in poem if len(line) > 0]
print(corpus_lines[0])

dear writers i’m compiling the first in what i hope is a series of publications i’m calling artists among artists the theme for issue is “faggot dinosaur” i hope to hear from you thank you and best wishes i think that i shall never fear a brontosaurus that is queer iguanodon as fetisheer a mammoth bringing up the rear an astrodon with extra gear metrosexual squirrel and deer a breeder with a dance career a fruit with cauliflower ear a lesbianic chanticleer a grinning limpishwristed lear the weird one or the mutineer but those who perfectly adhere stay clear stay clear stay clear stay clear


In [29]:
print(len(corpus_lines))
print(corpus_lines[0])
corpus_string = ' '.join(corpus_lines)

15650
dear writers i’m compiling the first in what i hope is a series of publications i’m calling artists among artists the theme for issue is “faggot dinosaur” i hope to hear from you thank you and best wishes i think that i shall never fear a brontosaurus that is queer iguanodon as fetisheer a mammoth bringing up the rear an astrodon with extra gear metrosexual squirrel and deer a breeder with a dance career a fruit with cauliflower ear a lesbianic chanticleer a grinning limpishwristed lear the weird one or the mutineer but those who perfectly adhere stay clear stay clear stay clear stay clear


In [30]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(corpus_lines)
# input_sequences = tokenizer.texts_to_sequences(corpus_lines)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
for line in corpus_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        

150002


In [31]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

MemoryError: Unable to allocate 237. GiB for an array with shape (3387937, 18745) and data type int32

In [15]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

MemoryError: Unable to allocate 307. GiB for an array with shape (587218, 140126) and data type float32

In [11]:
model = Sequential()
model.add(Embedding(total_words, 80, input_length=max_sequence_length-1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50))
model.add(tf.keras.layers.Dropout(0.1))
model.add(Dense(total_words/20))
model.add(Dense(total_words, activation='softmax'))
model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[28278,565562] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]